In [1]:
import pandas as pd
import numpy as np

In [2]:
def animals():
    a = pd.read_csv('AustinAnimalWorld.csv')
    a.columns = a.columns.str.lower().str.replace(' ','_')
    a.sex_upon_outcome = a.sex_upon_outcome.fillna('Unknown')
    a = a.drop(columns=['monthyear'])
    a.age_upon_outcome = a.age_upon_outcome.astype(str)

    days = []
    weeks = []
    months = []
    years = []

    for (listt,name, mult) in [(days,'day', 1), (weeks,'week', 7), (months,'month', 30), (years,'year', 365)]:
        for row in a.age_upon_outcome:
            if name in row:
                listt.append(abs(int(row[:2].strip()) * mult))
            else:
                listt.append(0)

    a['days'] = np.array(days)
    a['weeks'] = np.array(weeks)
    a['months'] = np.array(months)
    a['years'] = np.array(years)

    age = a.days + a.weeks + a.months + a.years
    a['age'] = age

    a = a.drop(columns=['days','weeks', 'months', 'years', 'age_upon_outcome', 'datetime', 'animal_id', 'date_of_birth'])

    gender = []
    neut_spay = []

    for row in a.sex_upon_outcome.astype(str):
        if row == 'Unknown':
            gender.append('unk')
            neut_spay.append('unk')
        elif row[0] == 'I':
            if row[7] == 'F':
                gender.append('female')
                neut_spay.append(False)
            if row[7] == 'M':
                gender.append('male')
                neut_spay.append(False)
        elif row[0] == 'N':
            gender.append('male')
            neut_spay.append(True)
        elif row[0] == 'S':
            gender.append('female')
            neut_spay.append(True)

    a['gender'] = gender
    a['neut_spay'] = neut_spay

    a = a.drop(columns='sex_upon_outcome')

    new_outcomes = []

    for row in a.outcome_type:
        if row in ['Adoption','Transfer','Return to Owner', 'Euthanasia', 'Died']:
            new_outcomes.append(row)
        else:
            new_outcomes.append('Other')

    a.outcome_type = new_outcomes

    condition = []

    for row in a.outcome_subtype:
        if row == 'Rabies Risk':
            condition.append('Rabies Risk')
        elif row == 'Suffering':
            condition.append('Suffering')
        elif row == 'Aggressive':
            condition.append('Aggressive')
        elif row == 'Medical':
            condition.append('Medical')
        elif row == 'Underage':
            condition.append('Underage')
        elif row == 'Behavior':
            condition.append('Behavior')
        elif row == 'Under Investigation':
            condition.append('Under Investigation')
        else:
            condition.append('Normal')

    a['condition'] = condition

    mylist = a.name.isna()
    the_list = [not elem for elem in mylist]
    a.name = the_list

    a = a.drop(columns='outcome_subtype')

    mixed = []
    new_breed = []

    for row in a.breed:
        if 'Mix' in row:
            mixed.append(True)
            new_breed.append(row[:-4])
        else:
            mixed.append(False)
            new_breed.append(row)

    a['new_breed'] = new_breed

    breed1 = []
    breed2 = []

    for breed, mix in zip(a.new_breed, mixed):
        if '/' in breed:
            br = breed.split('/')
            breed1.append(br[0])
            breed2.append(br[1])
        else:
            if mix is True:
                breed1.append(breed)
                breed2.append('mix')
            else:
                breed1.append(breed)
                breed2.append('purebred')

    a['breed1'] = breed1
    a['breed2'] = breed2

    b2_filter = a.breed2.value_counts().head(25).index

    bre2 = []

    for breed in breed2:
        if breed not in b2_filter:
            bre2.append('Other')
        else:
            bre2.append(breed)

    a.breed2 = bre2

    b1 = a.breed1.value_counts() 
    b1_filter = (b1[b1>200]).index

    bre1 = []

    for breed in breed1:
        if breed not in b1_filter:
            bre1.append('Other')
        else:
            bre1.append(breed)

    a.breed1 = bre1

    a = a.drop(columns='breed')
    return a

In [3]:
animals()

,name,outcome_type,animal_type,color,age,gender,neut_spay,condition,new_breed,breed1,breed2
0,False,Adoption,Dog,Brown/White,180,male,True,Normal,Airedale Terrier,Other,purebred
1,True,Adoption,Dog,Tan,730,male,True,Normal,Pit Bull,Pit Bull,mix
2,True,Adoption,Dog,Sable,120,male,True,Normal,Labrador Retriever,Labrador Retriever,purebred
3,False,Transfer,Cat,Brown Tabby/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
4,False,Transfer,Cat,Black/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
...,...,...,...,...,...,...,...,...,...,...,...
103462,True,Return to Owner,Dog,White,1460,male,True,Normal,Bulldog,Other,mix
103463,False,Transfer,Cat,Orange/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
103464,False,Transfer,Cat,Orange/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
103465,False,Transfer,Cat,Orange/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix


In [107]:
a = pd.read_csv('AustinAnimalWorld.csv')
a.columns = a.columns.str.lower().str.replace(' ','_')
a.sex_upon_outcome = a.sex_upon_outcome.fillna('Unknown')

In [81]:
a.head(1)

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color
0,A798981,NaN,07/06/2019 07:21:00 PM,07/06/2019 07:21:00 PM,01/02/2019,Adoption,NaN,Dog,Neutered Male,6 months,Airedale Terrier,Brown/White


In [97]:
a.age_upon_outcome.str[0]

0         6
1         2
2         4
3         1
4         1
         ..
103462    4
103463    1
103464    1
103465    1
103466    2
Name: age_upon_outcome, Length: 103467, dtype: object

In [9]:
a.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color
0,A798981,NaN,07/06/2019 07:21:00 PM,07/06/2019 07:21:00 PM,01/02/2019,Adoption,NaN,Dog,Neutered Male,6 months,Airedale Terrier,Brown/White
1,A792935,Tyler,07/06/2019 07:19:00 PM,07/06/2019 07:19:00 PM,04/17/2017,Adoption,NaN,Dog,Neutered Male,2 years,Pit Bull Mix,Tan
2,A798879,*Ron,07/06/2019 06:52:00 PM,07/06/2019 06:52:00 PM,02/14/2019,Adoption,NaN,Dog,Neutered Male,4 months,Labrador Retriever,Sable
3,A799278,NaN,07/06/2019 06:38:00 PM,07/06/2019 06:38:00 PM,06/29/2019,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Brown Tabby/White
4,A799275,NaN,07/06/2019 06:37:00 PM,07/06/2019 06:37:00 PM,06/29/2019,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Black/White


In [109]:
a.age_upon_outcome = a.age_upon_outcome.astype(str)
days = []
weeks = []
months = []
years = []

for (listt,name, mult) in [(days,'day', 1), (weeks,'week', 7), (months,'month', 30), (years,'year', 365)]:
    for row in a.age_upon_outcome:
        if name in row:
            listt.append(abs(int(row[:2].strip()) * mult))
        else:
            listt.append(0)

In [111]:
a['days'] = np.array(days)
a['weeks'] = np.array(weeks)
a['months'] = np.array(months)
a['years'] = np.array(years)

age = a.days + a.weeks + a.months + a.years

In [16]:
age = a.days + a.weeks + a.months + a.years

In [112]:
a['age'] = age

In [114]:
a.age = np.where(a.age > 730, 1095, a.age)

In [18]:
a.head()

,animal_id,name,datetime,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,days,weeks,months,years,age
0,A798981,NaN,07/06/2019 07:21:00 PM,01/02/2019,Adoption,NaN,Dog,Neutered Male,6 months,Airedale Terrier,Brown/White,0,0,180,0,180
1,A792935,Tyler,07/06/2019 07:19:00 PM,04/17/2017,Adoption,NaN,Dog,Neutered Male,2 years,Pit Bull Mix,Tan,0,0,0,730,730
2,A798879,*Ron,07/06/2019 06:52:00 PM,02/14/2019,Adoption,NaN,Dog,Neutered Male,4 months,Labrador Retriever,Sable,0,0,120,0,120
3,A799278,NaN,07/06/2019 06:38:00 PM,06/29/2019,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Brown Tabby/White,0,7,0,0,7
4,A799275,NaN,07/06/2019 06:37:00 PM,06/29/2019,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Black/White,0,7,0,0,7


In [19]:
a = a.drop(columns=['days','weeks', 'months', 'years', 'age_upon_outcome', 'datetime', 'animal_id', 'date_of_birth'])

In [20]:
a.head()

,name,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,breed,color,age
0,NaN,Adoption,NaN,Dog,Neutered Male,Airedale Terrier,Brown/White,180
1,Tyler,Adoption,NaN,Dog,Neutered Male,Pit Bull Mix,Tan,730
2,*Ron,Adoption,NaN,Dog,Neutered Male,Labrador Retriever,Sable,120
3,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Brown Tabby/White,7
4,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Black/White,7


In [21]:
a.sex_upon_outcome.value_counts()

Neutered Male    36435
Spayed Female    32832
Intact Male      13010
Intact Female    12561
Unknown           8629
Name: sex_upon_outcome, dtype: int64

In [22]:
gender = []
neut_spay = []

for row in a.sex_upon_outcome.astype(str):
    if row == 'Unknown':
        gender.append('unk')
        neut_spay.append('unk')
    elif row[0] == 'I':
        if row[7] == 'F':
            gender.append('female')
            neut_spay.append(False)
        if row[7] == 'M':
            gender.append('male')
            neut_spay.append(False)
    elif row[0] == 'N':
        gender.append('male')
        neut_spay.append(True)
    elif row[0] == 'S':
        gender.append('female')
        neut_spay.append(True)

In [23]:
len(gender), len(neut_spay), len(a)

(103467, 103467, 103467)

In [24]:
a['gender'] = gender
a['net_spay'] = neut_spay

In [25]:
a.head(15)

,name,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,breed,color,age,gender,net_spay
0,NaN,Adoption,NaN,Dog,Neutered Male,Airedale Terrier,Brown/White,180,male,True
1,Tyler,Adoption,NaN,Dog,Neutered Male,Pit Bull Mix,Tan,730,male,True
2,*Ron,Adoption,NaN,Dog,Neutered Male,Labrador Retriever,Sable,120,male,True
3,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk
4,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Black/White,7,unk,unk
5,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Black/White,7,unk,unk
6,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Brown Tabby,7,unk,unk
7,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Orange Tabby/White,7,unk,unk
8,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair Mix,Tortie,7,unk,unk
9,NaN,Transfer,Partner,Cat,Unknown,Domestic Shorthair,Tortie,14,unk,unk


In [26]:
a = a.drop(columns='sex_upon_outcome')

In [27]:
a.outcome_type.value_counts()

Adoption           44882
Transfer           30778
Return to Owner    18680
Euthanasia          7296
Died                 957
Rto-Adopt            410
Disposal             378
Missing               61
Relocate              19
Name: outcome_type, dtype: int64

In [28]:
a[a.outcome_type == 'Adoption']

,name,outcome_type,outcome_subtype,animal_type,breed,color,age,gender,net_spay
0,NaN,Adoption,NaN,Dog,Airedale Terrier,Brown/White,180,male,True
1,Tyler,Adoption,NaN,Dog,Pit Bull Mix,Tan,730,male,True
2,*Ron,Adoption,NaN,Dog,Labrador Retriever,Sable,120,male,True
12,NaN,Adoption,Foster,Dog,Cocker Spaniel/Australian Terrier,Tan/White,60,female,True
14,Minnie,Adoption,NaN,Dog,Chihuahua Shorthair,Brown,2555,female,True
...,...,...,...,...,...,...,...,...,...
103433,Gracie,Adoption,NaN,Dog,Labrador Retriever Mix,Black/White,1095,female,True
103456,Blakie,Adoption,NaN,Cat,Domestic Shorthair Mix,Tortie,180,female,True
103457,Jake,Adoption,NaN,Cat,Domestic Shorthair Mix,Black,150,male,True
103459,*Starla,Adoption,NaN,Dog,Anatol Shepherd Mix,White/Brown,1095,female,True


In [29]:
a[a.outcome_type == 'Adoption'].outcome_subtype.value_counts()

Foster     7955
Offsite     379
Barn          1
Name: outcome_subtype, dtype: int64

In [30]:
a.animal_type.value_counts()

Dog          59000
Cat          38520
Other         5453
Bird           478
Livestock       16
Name: animal_type, dtype: int64

In [31]:
bird = a[a.animal_type == 'Bird']
dog = a[a.animal_type == 'Dog']
cat = a[a.animal_type == 'Cat']
other = a[(a.animal_type == 'Livestock') | (a.animal_type == 'Other')]

In [32]:
other

,name,outcome_type,outcome_subtype,animal_type,breed,color,age,gender,net_spay
90,*Keto,Adoption,NaN,Other,American Sable,Brown,180,male,True
101,NaN,Euthanasia,Rabies Risk,Other,Fox,Gray/Black,120,unk,unk
114,*Jim,Adoption,NaN,Other,American,Black,365,male,True
140,NaN,Euthanasia,Rabies Risk,Other,Bat,Brown,730,unk,unk
141,NaN,Euthanasia,Rabies Risk,Other,Raccoon,Black/White,730,unk,unk
...,...,...,...,...,...,...,...,...,...
103342,NaN,Euthanasia,Rabies Risk,Other,Raccoon Mix,Black/White,730,unk,unk
103346,NaN,Euthanasia,Rabies Risk,Other,Skunk,Black/White,270,unk,unk
103348,NaN,Transfer,Partner,Livestock,Miniature,Liver/Cream,1825,male,False
103363,NaN,Euthanasia,Rabies Risk,Other,Raccoon Mix,Brown,180,unk,unk


In [33]:
a.head()

,name,outcome_type,outcome_subtype,animal_type,breed,color,age,gender,net_spay
0,NaN,Adoption,NaN,Dog,Airedale Terrier,Brown/White,180,male,True
1,Tyler,Adoption,NaN,Dog,Pit Bull Mix,Tan,730,male,True
2,*Ron,Adoption,NaN,Dog,Labrador Retriever,Sable,120,male,True
3,NaN,Transfer,Partner,Cat,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk
4,NaN,Transfer,Partner,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk


In [34]:
a.outcome_type.value_counts()

Adoption           44882
Transfer           30778
Return to Owner    18680
Euthanasia          7296
Died                 957
Rto-Adopt            410
Disposal             378
Missing               61
Relocate              19
Name: outcome_type, dtype: int64

In [35]:
new_outcomes = []

for row in a.outcome_type:
    if row in ['Adoption','Transfer','Return to Owner', 'Euthanasia', 'Died']:
        new_outcomes.append(row)
    else:
        new_outcomes.append('Other')

In [36]:
a.outcome_type = new_outcomes

In [37]:
a.outcome_type.value_counts()

Adoption           44882
Transfer           30778
Return to Owner    18680
Euthanasia          7296
Died                 957
Other                874
Name: outcome_type, dtype: int64

In [38]:
a[a.outcome_type == 'Adoption'].outcome_subtype.value_counts()

Foster     7955
Offsite     379
Barn          1
Name: outcome_subtype, dtype: int64

In [39]:
a[a.outcome_type == 'Transfer'].outcome_subtype.value_counts()

Partner    25766
SCRP        3211
Snr         1798
Barn           3
Name: outcome_subtype, dtype: int64

In [40]:
a[a.outcome_type == 'Return to Owner'].outcome_subtype.value_counts()

Field         4
Foster        2
Customer S    1
At Vet        1
Name: outcome_subtype, dtype: int64

In [41]:
a[a.outcome_type == 'Euthanasia'].outcome_subtype.value_counts()

Rabies Risk            3116
Suffering              2897
Aggressive              521
Medical                 297
Behavior                148
At Vet                   67
Underage                 29
Court/Investigation      23
Name: outcome_subtype, dtype: int64

In [42]:
a[a.outcome_type == 'Other'].outcome_subtype.value_counts()

In Foster         22
Possible Theft    13
In Kennel         11
Name: outcome_subtype, dtype: int64

In [43]:
a[a.outcome_type == 'Euthanasia'].outcome_subtype.value_counts()

Rabies Risk            3116
Suffering              2897
Aggressive              521
Medical                 297
Behavior                148
At Vet                   67
Underage                 29
Court/Investigation      23
Name: outcome_subtype, dtype: int64

In [44]:
len(a.outcome_subtype)

103467

In [45]:
condition = []

for row in a.outcome_subtype:
    if row == 'Rabies Risk':
        condition.append('Rabies Risk')
    elif row == 'Suffering':
        condition.append('Suffering')
    elif row == 'Aggressive':
        condition.append('Aggressive')
    elif row == 'Medical':
        condition.append('Medical')
    elif row == 'Underage':
        condition.append('Underage')
    elif row == 'Behavior':
        condition.append('Behavior')
    elif row == 'Under Investigation':
        condition.append('Under Investigation')
    else:
        condition.append('Normal')
    

In [46]:
len(condition)

103467

In [47]:
a['condition'] = condition

In [48]:
a.head()

,name,outcome_type,outcome_subtype,animal_type,breed,color,age,gender,net_spay,condition
0,NaN,Adoption,NaN,Dog,Airedale Terrier,Brown/White,180,male,True,Normal
1,Tyler,Adoption,NaN,Dog,Pit Bull Mix,Tan,730,male,True,Normal
2,*Ron,Adoption,NaN,Dog,Labrador Retriever,Sable,120,male,True,Normal
3,NaN,Transfer,Partner,Cat,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk,Normal
4,NaN,Transfer,Partner,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk,Normal


In [49]:
mylist = a.name.isna()
the_list = [not elem for elem in mylist]
a.name = the_list

In [50]:
a.head()

,name,outcome_type,outcome_subtype,animal_type,breed,color,age,gender,net_spay,condition
0,False,Adoption,NaN,Dog,Airedale Terrier,Brown/White,180,male,True,Normal
1,True,Adoption,NaN,Dog,Pit Bull Mix,Tan,730,male,True,Normal
2,True,Adoption,NaN,Dog,Labrador Retriever,Sable,120,male,True,Normal
3,False,Transfer,Partner,Cat,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk,Normal
4,False,Transfer,Partner,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk,Normal


In [51]:
a = a.drop(columns='outcome_subtype')

In [52]:
a.head()

,name,outcome_type,animal_type,breed,color,age,gender,net_spay,condition
0,False,Adoption,Dog,Airedale Terrier,Brown/White,180,male,True,Normal
1,True,Adoption,Dog,Pit Bull Mix,Tan,730,male,True,Normal
2,True,Adoption,Dog,Labrador Retriever,Sable,120,male,True,Normal
3,False,Transfer,Cat,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk,Normal
4,False,Transfer,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk,Normal


In [53]:
a.breed[:50]

0                           Airedale Terrier
1                               Pit Bull Mix
2                         Labrador Retriever
3                     Domestic Shorthair Mix
4                     Domestic Shorthair Mix
5                     Domestic Shorthair Mix
6                     Domestic Shorthair Mix
7                     Domestic Shorthair Mix
8                     Domestic Shorthair Mix
9                         Domestic Shorthair
10                        Domestic Shorthair
11                      Domestic Medium Hair
12         Cocker Spaniel/Australian Terrier
13                        Domestic Shorthair
14                       Chihuahua Shorthair
15                             Cairn Terrier
16                       German Shepherd Mix
17               Labrador Retriever/Pit Bull
18                            Border Terrier
19                        Domestic Shorthair
20         Cocker Spaniel/Australian Terrier
21                    Domestic Shorthair Mix
22        

In [54]:
a.breed[1][-4:]

' Mix'

In [55]:
bird = a[a.animal_type == 'Bird']
dog = a[a.animal_type == 'Dog']
cat = a[a.animal_type == 'Cat']
other = a[(a.animal_type == 'Livestock') | (a.animal_type == 'Other')]

In [56]:
dog.breed.value_counts().tail(500)

Pekingese/Lhasa Apso                      1
Pembroke Welsh Corgi/Pit Bull             1
Border Collie/Great Dane                  1
Collie Smooth/Jack Russell Terrier        1
Anatol Shepherd/Belgian Malinois          1
                                         ..
Jack Russell Terrier/Anatol Shepherd      1
Dachshund Longhair/Pomeranian             1
German Shorthair Pointer/Border Collie    1
Miniature Pinscher/Pomeranian             1
Pit Bull/Border Terrier                   1
Name: breed, Length: 500, dtype: int64

In [57]:
mixed = []
new_breed = []

for row in a.breed:
    if 'Mix' in row:
        mixed.append(True)
        new_breed.append(row[:-4])
    else:
        mixed.append(False)
        new_breed.append(row)
    
a['new_breed'] = new_breed

In [58]:
breed1 = []
breed2 = []

for breed, mix in zip(a.new_breed, mixed):
    if '/' in breed:
        br = breed.split('/')
        breed1.append(br[0])
        breed2.append(br[1])
    else:
        if mix is True:
            breed1.append(breed)
            breed2.append('mix')
        else:
            breed1.append(breed)
            breed2.append('purebred')
            
a['breed1'] = breed1
a['breed2'] = breed2

In [59]:
a.breed1.value_counts()

Domestic Shorthair              31267
Pit Bull                         8800
Labrador Retriever               8167
Chihuahua Shorthair              7147
German Shepherd                  3678
                                ...  
Grand Basset Griffon Vendeen        1
African                             1
Chinchilla-Amer                     1
Wren                                1
Miniature                           1
Name: breed1, Length: 357, dtype: int64

In [60]:
a.breed2.value_counts()

mix                    85073
purebred                8683
Labrador Retriever      1321
Chihuahua Shorthair      716
Pit Bull                 654
                       ...  
Abyssinian                 1
Bearded Collie             1
Kangal                     1
Skye Terrier               1
Lop-Mini                   1
Name: breed2, Length: 190, dtype: int64

In [61]:
a.head(15)

,name,outcome_type,animal_type,breed,color,age,gender,net_spay,condition,new_breed,breed1,breed2
0,False,Adoption,Dog,Airedale Terrier,Brown/White,180,male,True,Normal,Airedale Terrier,Airedale Terrier,purebred
1,True,Adoption,Dog,Pit Bull Mix,Tan,730,male,True,Normal,Pit Bull,Pit Bull,mix
2,True,Adoption,Dog,Labrador Retriever,Sable,120,male,True,Normal,Labrador Retriever,Labrador Retriever,purebred
3,False,Transfer,Cat,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
4,False,Transfer,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
5,False,Transfer,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
6,False,Transfer,Cat,Domestic Shorthair Mix,Brown Tabby,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
7,False,Transfer,Cat,Domestic Shorthair Mix,Orange Tabby/White,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
8,False,Transfer,Cat,Domestic Shorthair Mix,Tortie,7,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,mix
9,False,Transfer,Cat,Domestic Shorthair,Tortie,14,unk,unk,Normal,Domestic Shorthair,Domestic Shorthair,purebred


In [62]:
a = a.drop(columns='new_breed')

In [63]:
a.breed1.value_counts().head(100)

Domestic Shorthair          31267
Pit Bull                     8800
Labrador Retriever           8167
Chihuahua Shorthair          7147
German Shepherd              3678
                            ...  
Vizsla                         81
German Shorthair Pointer       79
Collie Rough                   78
Shetland Sheepdog              76
Wire Hair Fox Terrier          75
Name: breed1, Length: 100, dtype: int64

In [64]:
b2_filter = a.breed2.value_counts().head(25).index
b2_filter

Index(['mix', 'purebred', 'Labrador Retriever', 'Chihuahua Shorthair',
       'Pit Bull', 'German Shepherd', 'Dachshund', 'Australian Cattle Dog',
       'Miniature Poodle', 'Beagle', 'Border Collie', 'Boxer',
       'Great Pyrenees', 'Catahoula', 'Tan Hound', 'Rat Terrier',
       'Jack Russell Terrier', 'Pointer', 'Chow Chow', 'Chinese Sharpei',
       'Pug', 'Yorkshire Terrier', 'Australian Shepherd',
       'Cardigan Welsh Corgi', 'Rottweiler'],
      dtype='object')

In [65]:
a.breed2.value_counts().head(25)

mix                      85073
purebred                  8683
Labrador Retriever        1321
Chihuahua Shorthair        716
Pit Bull                   654
German Shepherd            517
Dachshund                  497
Australian Cattle Dog      385
Miniature Poodle           322
Beagle                     320
Border Collie              281
Boxer                      275
Great Pyrenees             194
Catahoula                  184
Tan Hound                  152
Rat Terrier                149
Jack Russell Terrier       144
Pointer                    142
Chow Chow                  141
Chinese Sharpei            133
Pug                        128
Yorkshire Terrier          123
Australian Shepherd        105
Cardigan Welsh Corgi       103
Rottweiler                  98
Name: breed2, dtype: int64

In [66]:
b2_filter = a.breed2.value_counts().head(25).index

bre2 = []

for breed in breed2:
    if breed not in b2_filter:
        bre2.append('Other')
    else:
        bre2.append(breed)

In [67]:
pd.DataFrame(bre2).value_counts()

mix                      85073
purebred                  8683
Other                     2627
Labrador Retriever        1321
Chihuahua Shorthair        716
Pit Bull                   654
German Shepherd            517
Dachshund                  497
Australian Cattle Dog      385
Miniature Poodle           322
Beagle                     320
Border Collie              281
Boxer                      275
Great Pyrenees             194
Catahoula                  184
Tan Hound                  152
Rat Terrier                149
Jack Russell Terrier       144
Pointer                    142
Chow Chow                  141
Chinese Sharpei            133
Pug                        128
Yorkshire Terrier          123
Australian Shepherd        105
Cardigan Welsh Corgi       103
Rottweiler                  98
dtype: int64

In [68]:
a.breed2 = bre2

In [69]:
a.breed2

0         purebred
1              mix
2         purebred
3              mix
4              mix
            ...   
103462         mix
103463         mix
103464         mix
103465         mix
103466         mix
Name: breed2, Length: 103467, dtype: object

In [70]:
b1 = a.breed1.value_counts() 

In [71]:
b1_filter = (b1[b1>200]).index

In [72]:
b1 = a.breed1.value_counts() 
b1_filter = (b1[b1>200]).index

bre1 = []

for breed in breed1:
    if breed not in b1_filter:
        bre1.append('Other')
    else:
        bre1.append(breed)
        
a.breed1 = bre1

In [73]:
a.breed1 = bre1

In [74]:
a.breed1.value_counts() 

Domestic Shorthair                31267
Other                              8953
Pit Bull                           8800
Labrador Retriever                 8167
Chihuahua Shorthair                7147
German Shepherd                    3678
Domestic Medium Hair               3297
Bat                                2716
Australian Cattle Dog              1833
Domestic Longhair                  1573
Dachshund                          1501
Siamese                            1323
Boxer                              1254
Border Collie                      1222
Miniature Poodle                   1047
Siberian Husky                      941
Beagle                              880
Raccoon                             826
Australian Shepherd                 814
Yorkshire Terrier                   809
Catahoula                           806
Rat Terrier                         771
Miniature Schnauzer                 759
Jack Russell Terrier                756
Great Pyrenees                      752


In [75]:
a.head()

,name,outcome_type,animal_type,breed,color,age,gender,net_spay,condition,breed1,breed2
0,False,Adoption,Dog,Airedale Terrier,Brown/White,180,male,True,Normal,Other,purebred
1,True,Adoption,Dog,Pit Bull Mix,Tan,730,male,True,Normal,Pit Bull,mix
2,True,Adoption,Dog,Labrador Retriever,Sable,120,male,True,Normal,Labrador Retriever,purebred
3,False,Transfer,Cat,Domestic Shorthair Mix,Brown Tabby/White,7,unk,unk,Normal,Domestic Shorthair,mix
4,False,Transfer,Cat,Domestic Shorthair Mix,Black/White,7,unk,unk,Normal,Domestic Shorthair,mix


In [76]:
a = a.drop(columns='breed')

In [77]:
a.color.value_counts().head(30)

Black/White            10800
Black                   8627
Brown Tabby             5850
Brown                   4460
White                   3680
Brown/White             3339
Tan/White               3203
Brown Tabby/White       3034
Orange Tabby            2848
White/Black             2811
Blue/White              2799
Tan                     2615
Tricolor                2476
Black/Tan               2381
Black/Brown             2157
White/Brown             2033
Brown/Black             1823
Brown Brindle/White     1787
Blue                    1782
Tortie                  1768
Calico                  1756
White/Tan               1592
Blue Tabby              1486
Orange Tabby/White      1403
Red                     1314
Torbie                  1133
Red/White               1113
Tan/Black                927
Brown Brindle            913
Chocolate/White          786
Name: color, dtype: int64

In [78]:
a.to_csv('animals.csv', index=False)